# Imports

In [1]:
import pandas as pd
import ipywidgets as widgets
import plotly.express as px
from ipywidgets import fixed
from matplotlib import pyplot as plt
from geopy.geocoders import Nominatim
from matplotlib import gridspec

# Reading Data

In [2]:
pd.set_option('display.max_columns', 25)
df = pd.read_csv('C:/Users/Ayron/repos/Datasets/kc_house_data.csv')

# 1. Qual a média do preço de compra dos imóveis por "Nível"?


In [3]:
# Nível 0 => Preço entre $ 0 e $ 321.950
# Nível 1 => Preço entre $ 321.950 e $ 450.000
# Nível 2 => Preço entre $ 450.000 e $ 645.000
# Nível 3 => Acima $ 645.000

In [4]:
for i in range(len(df)):
    if (df.loc[i, 'price'] > 0) & (df.loc[i, 'price'] < 321950):
        df.loc[i, 'level'] = 0
    elif (df.loc[i, 'price'] >= 321950) & (df.loc[i, 'price'] < 450000):
        df.loc[i, 'level'] = 1
    elif (df.loc[i, 'price'] >= 450000) & (df.loc[i, 'price'] < 645000):
        df.loc[i, 'level'] = 2
    else:
        df.loc[i, 'level'] = 3

In [5]:
df[['price', 'level']].groupby('level').mean().reset_index()

,level,price
0,0.0,251544.621507
1,1.0,383585.205332
2,2.0,536164.422840
3,3.0,985008.983189


# 2. Qual a média do tamanho da sala de estar dos imóveis por "Size"?
* Size 0 => Tamanho entre 0 e 1427 sqft
* Size 1 => Tamanho entre 1427 e 1910 sqft
* Size 2 => Tamanho entre 1910 e 2550 sqft
* Size 3 => Tamanho acima de 2550 sqft

In [6]:
for i in range(len(df)):
    if (df.loc[i, 'sqft_living'] > 0) & (df.loc[i, 'sqft_living'] < 1427):
        df.loc[i, 'size_living'] = 0
    elif (df.loc[i, 'sqft_living'] >= 1427) & (df.loc[i, 'sqft_living'] < 1910):
        df.loc[i, 'size_living'] = 1
    elif (df.loc[i, 'sqft_living'] >= 1910) & (df.loc[i, 'sqft_living'] < 2550):
        df.loc[i, 'size_living'] = 2
    else:
        df.loc[i, 'sqft_living'] = 3

In [7]:
df[['sqft_living', 'size_living']].groupby('size_living').mean().reset_index()

,size_living,sqft_living
0,0.0,1123.776421
1,1.0,1661.198088
2,2.0,2202.700239


# 3. Adicione as seguintes informações ao conjunto de dados original:
* Place ID: Identificação da localização
* OSM Type: Open Street Map Type
* Country: Nome do País
* Country Code: Código do País

In [8]:
#Inicializando Nominatim API
geolocator = Nominatim(user_agent='geoapiExercises')
response = geolocator.reverse('47.5112,-122.257')
response.raw

{'place_id': 148421265,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'osm_type': 'way',
 'osm_id': 236673600,
 'lat': '47.5112302',
 'lon': '-122.25676111324441',
 'display_name': '10012, 61st Avenue South, Rainier Beach, Seattle, King County, Washington, 98178, United States',
 'address': {'house_number': '10012',
  'road': '61st Avenue South',
  'neighbourhood': 'Rainier Beach',
  'city': 'Seattle',
  'county': 'King County',
  'state': 'Washington',
  'postcode': '98178',
  'country': 'United States',
  'country_code': 'us'},
 'boundingbox': ['47.511189', '47.5112943', '-122.2568571', '-122.2566651']}

In [9]:
#Creaty empty rows
df['Place_ID'] = 'NA'
df['OSM_TypeID'] = 'NA'
df['Country'] = 'NA'
df['Country_Code'] = 'NA'

In [10]:
#Inicializando Nominatim API
geolocator = Nominatim(user_agent='geoapiExercises')

query = str(df.loc[i, 'lat']) + ',' + str(df.loc[i, 'long'])
response = geolocator.reverse(query)

     
df.loc[i, 'Place_ID'] = response.raw['place_id']
df.loc[i, 'OSM_TypeID'] = response.raw['osm_type']
df.loc[i, 'Country'] = response.raw['address']['country']
df.loc[i, 'Country_Code'] = response.raw['address']['country_code']

In [11]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 27)
df.head(2)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,level,size_living,Place_ID,OSM_TypeID,Country,Country_Code
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650,0.0,0.0,NA,NA,NA,NA
1,6414100192,20141209T000000,538000.0,3,2.25,3,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639,2.0,NaN,NA,NA,NA,NA


# 4.0 Adicione os seguintes filtros no Mapa:
* Tamanho mínimo da área da sala de estar.
* Número mínimo de banheiros.
* Valor máximo do Preço.
* Tamanho máximo da área do porão.
* Filtro das consdições do imóvel.
* Filtro por ano de construção.

In [12]:
#Interactive buttons
style = {'description_width':'initial'}

area_limit = widgets.IntSlider(value=1240, min=3, max=2547, 
                               step=1, description='Maximum Area', 
                               disable=False, style=style)

bathrooms_limit = widgets.IntSlider(value=2, min=0, max=8, 
                               step=1, description='Bathrooms', 
                               disable=False, style=style)

price_limit = widgets.IntSlider(value=450000, min=75000, max=7700000, 
                               step=1, description='Maximum Price', 
                               disable=False, style=style)

basement_limit = widgets.IntSlider(value=550, min=0, max=4820, 
                               step=1, description='Maximum Basement', 
                               disable=False, style=style)

condition_limit = widgets.IntSlider(value=3, min=1, max=5, 
                               step=1, description='Maximum Condition House', 
                               disable=False, style=style)

year_limit = widgets.IntSlider(value=1958, min=1900, max=2015, 
                               step=1, description='Maximum Year Built', 
                               disable=False, style=style)

In [13]:
def update_map(df, limit1, limit2, limit3, limit4, condition, construction):

    houses = df[(df['sqft_living'] >= limit1) & (df['bathrooms'] >= limit2) &
                (df['price'] <= limit3) & (df['sqft_basement'] <= limit4) &
                (df['condition'] >= condition) & (df['yr_built'] <= construction)][['id', 'lat', 
                'long', 'sqft_living', 'bathrooms', 'price', 'sqft_basement', 'condition', 'yr_built', 'level']]
    
    fig = px.scatter_mapbox(houses,
                           lat='lat',
                           lon='long',
                           color='level',
                           size='price',
                           color_continuous_scale=px.colors.cyclical.IceFire,
                           size_max=15,
                           zoom=10)
    fig.update_layout(mapbox_style='open-street-map')
    fig.update_layout(height=600, margin={'r':0, 't':0, 'l':0, 'b':0})
    fig.show()

In [14]:
widgets.interactive(update_map, df=fixed(df), limit1=area_limit, limit2=bathrooms_limit, limit3=price_limit,
                   limit4=basement_limit, condition=condition_limit, construction=year_limit)

interactive(children=(IntSlider(value=1240, description='Maximum Area', max=2547, min=3, style=SliderStyle(des…

# 5. Adicione os seguintes filtros no Dashboard:

* Filtro por data disponível para compra
* Filtro por ano de renovação
* Filtro se possui vista para água ou não.

In [16]:
#data = pd.read_csv('C:/Users/Ayron/repos/Datasets/kc_house_data.csv')

df['year'] = pd.to_datetime(df['date']).dt.strftime('%Y')
df['date'] = pd.to_datetime(df['date']).dt.strftime('%Y-%m-%d')
df['year_week'] = pd.to_datetime(df['date']).dt.strftime('%Y-%m-%U')

In [23]:
#Widgets control

date_limit = widgets.SelectionSlider(options=df['date'].sort_values().unique().tolist(), 
                               value='2014-11-30', 
                               description='Disponível', 
                               disable=False, 
                               continuous_update=False, 
                               orientation='horizontal', 
                               readout=True)

renovated_limit = widgets.BoundedIntText(value=1943,
                                         min=1930,
                                         max=2015,
                                         step=1,
                                         description='Ano de Renovação',
                                         disabled=False)

df['is_waterfront'] = df['waterfront'].apply(lambda x:'yes' if x == 1 else 'no')

waterfront_limit = widgets.Dropdown(options= df['is_waterfront'].unique().tolist(), 
                                    value='yes', description='Water View', disable=False)

In [24]:
def update_map(df, limit, option, year):
    
    #Filter data
    
    df_2 = df[(df['date'] >= limit) & (df['is_waterfront'] == option) & 
              (df['yr_renovated'] >= year)].copy()
    
    fig = plt.figure(figsize=(21,12))
              
    specs = gridspec.GridSpec(ncols=2, nrows=2, figure=fig)
    
    ax1 = fig.add_subplot(specs[0, :]) # First rows
    ax2 = fig.add_subplot(specs[1, 0]) # Second Row First Column
    ax3 = fig.add_subplot(specs[1, 1]) # Second Row Second Column
    
    by_year = df_2[['id', 'year',]].groupby('year').sum().reset_index()
    ax1.bar(by_year['year'], by_year['id'])
    
    by_day = df_2[['id', 'date']].groupby('date').mean().reset_index()
    ax2.plot(by_day['date'], by_day['id'])
    ax2.set_title('title: Avg Price by Day')

    by_week_of_year = df_2[['id', 'year_week']].groupby('year_week').mean().reset_index()
    ax3.bar(by_week_of_year['year_week'], by_week_of_year['id'])
    ax3.set_title('title: Avg Price by Week Of Year')
    plt.xticks(rotation=60);

In [25]:
widgets.interactive(update_map, df=fixed(df), limit=date_limit, option=waterfront_limit, year=renovated_limit)

interactive(children=(SelectionSlider(continuous_update=False, description='Disponível', index=211, options=('…